Importiere die requests Library

In [1]:
import requests

Importiere die Pandas Library in der Kurzform.

In [2]:
import pandas as pd

1. Unter Verwendung der [USGS.gov Earthquake API](https://earthquake.usgs.gov) ziehe heraus, wieviele Erdbeben sich im August 2018 ereignet haben?

In [3]:
datum1 = "2018-08-01"
datum2 = "2018-08-31"

url_start = "https://earthquake.usgs.gov/fdsnws/event/1/count?starttime="
url_middle = "&endtime="

erdbeben = requests.get(url_start + datum1 + url_middle + datum2)

print(erdbeben.text)

17045


2. Mache das gleiche für August 2017, 2016, 2015?

In [4]:
datum1 = "2017-08-01"
datum2 = "2017-08-31"

erdbeben = requests.get(url_start + datum1 + url_middle + datum2)

print(erdbeben.text)

12300


In [5]:
datum1 = "2016-08-01"
datum2 = "2016-08-31"

erdbeben = requests.get(url_start + datum1 + url_middle + datum2)

print(erdbeben.text)

10878


In [6]:
datum1 = "2015-08-01"
datum2 = "2015-08-31"

erdbeben = requests.get(url_start + datum1 + url_middle + datum2)

print(erdbeben.text)

9981


#### Retos kompakte Lösung:

In [7]:
# https://earthquake.usgs.gov/fdsnws/event/1/count?starttime=2014-01-01&endtime=2014-01-02
import requests
import pandas as pd
base = "https://earthquake.usgs.gov/fdsnws/event/1/"
action = "count?"
# Url = "starttime=2018-09-25&endtime=2018-09-30-25&endtime=2018-09-30"
for jahr in range(2015, 2019):
    
    url = base + action + "starttime=" + str(jahr) + "-08-01&endtime=" + str(jahr) + "-08-31"
    
    response = requests.get(url)
    print(str(jahr) + ": " + str(response.text))

2015: 9981
2016: 10878
2017: 12300
2018: 17045


3. Zeige nur die Erdbeben an, die in den letzten 5 Jahren eine Mindeststärke von 5,5 hatten? (Neue Parameter kannst Du mit den Zeichen "&" ergänzen. Lese Dich in die Dokumentation ein, um noch mehr Parameter kennezulernen)

#### Mein Versuch:
Zwei Probleme: 1. Ich frage sämtliche Beben ab (ohne Magnituden-Beschränkung) und überlaste damit das System. 2. Ich wandle immer wieder im Loop drin den requesteten Text per Json um in Dictionarys, statt das am Anfang zu machen.

In [ ]:
erdbeben = requests.get("https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2014-09-23&endtime=2014-09-25")

liste = []

for beben in range(0, int(len(erdbeben.json()["features"]))):
    magnitude = erdbeben.json()["features"][int(beben)]["properties"]["mag"]
      
    if float(magnitude) >= 5.5:
        place = erdbeben.json()["features"][int(beben)]["properties"]["place"]
        time = erdbeben.json()["features"][int(beben)]["properties"]["time"]
        
        mini_dict = {"Ort": place, "Zeit": time, "Stärke": magnitude}
  
        liste.append(mini_dict)

pd.DataFrame(liste)

#### Step-by-step-Verbesserung meines Versuchs:
(scheitert an Datenmenge)

In [62]:
### befehl .keys() zeigt Keys eines Dictionarys an

erdbeben = requests.get("https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2014-09-23&endtime=2014-09-25")
erdbeben = erdbeben.json()
erdbeben.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

In [40]:
# machen wir eine Liste mit den einzelnen Erdbeben und erledigen schon die ganze json-Transormation ein für allemal:
erdbeben = requests.get("https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2014-09-24&endtime=2019-09-24")
erdbeben = erdbeben.json()
erdbeben.keys()
erdbebenliste = erdbeben['features']

leereliste = []

for erdbeben in erdbebenliste:
    mag = erdbeben['properties']['mag']
    place = erdbeben['properties']['place']
    time = erdbeben['properties']['time']
    ### print(mag) ->> hier haben wir zur Fehlerfindung die Ergebnisse ausgedruckt.
    
    ### hier gehts darum, einen offensichtlich bestehenden Fehler bei den Magnituden zu umgehen
    try:
        if mag >= 5.5:
            mini_dict = {"Ort": place, "Zeit": time, "Stärke": mag}
            leereliste.append(mini_dict)
    except:
        "SyntaxError"

pd.DataFrame(leereliste)[-10:]

### Problem: Die Datenmenge ist zu gross, get nicht. Deshalb Lösung mit 

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

#### Die überlegene Lösung: im Request schon die Magnitude eingeben

In [52]:
erdbeben = requests.get("https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2014-09-24&endtime=2019-09-24&minmagnitude=5.5")
erdbeben = erdbeben.json()
erdbeben.keys()
erdbebenliste = erdbeben['features']

leereliste = []

for erdbeben in erdbebenliste:
    mag = erdbeben['properties']['mag']
    place = erdbeben['properties']['place']
    time = erdbeben['properties']['time']
    
    mini_dict = {"Ort": place, "Zeit": time, "Stärke": mag}
    leereliste.append(mini_dict)
 
pd.DataFrame(leereliste)[-10:]

,Ort,Stärke,Zeit
2222,"71km S of Paratunka, Russia",5.7,1412254625750
2223,"16km N of Kandrian, Papua New Guinea",6.0,1412134731760
2224,"265km SE of Lambasa, Fiji",5.5,1412116467030
2225,Carlsberg Ridge,5.5,1412095556230
2226,"105km WNW of Hofn, Iceland",5.6,1411998181660
2227,"221km WNW of Pangai, Tonga",5.7,1411885415750
2228,"285km ENE of Port Blair, India",5.5,1411705284080
2229,"96km WNW of Willow, Alaska",6.2,1411667477606
2230,"157km SSW of Gizo, Solomon Islands",6.1,1411636430000
2231,"56km NW of San Antonio de los Cobres, Argentina",6.2,1411557375010


4. Zeige eine Liste der 10 stärksten Erdbeben des laufenden Jahres an. (Nicht vergessen, nun musst Du die Methode wechseln. Nicht mehr "count", sondern "query").

In [ ]:
### Versuch mit if-Schleife: neue Beben nur in liste, wenn Magnitude grösser als letztes

erdbeben = requests.get("https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2019-01-01&endtime=2019-01-02")

liste = []
magnitude_last = 0

for beben in range(0, int(len(erdbeben.json()["features"]))):
    magnitude = erdbeben.json()["features"][int(beben)]["properties"]["mag"]
    place = erdbeben.json()["features"][int(beben)]["properties"]["place"]
    time = erdbeben.json()["features"][int(beben)]["properties"]["time"]
                   
    mini_dict = {"Ort": place, "Zeit": time, "Stärke": magnitude}
    if int(magnitude) > int(magnitude_last):
        liste.append(mini_dict)
        magnitude_last = magnitude
    else:
        continue

pd.DataFrame(liste)[-10:]

#### Neuer Anlauf

In [22]:
erdbeben = requests.get("https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2019-01-01&endtime=2019-09-23")

In [24]:
erdbeben = requests.get("https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2019-01-01&endtime=2019-01-02")

liste = []

for beben in range(0, int(len(erdbeben.json()["features"]))):
    magnitude = erdbeben.json()["features"][int(beben)]["properties"]["mag"]
    place = erdbeben.json()["features"][int(beben)]["properties"]["place"]
    time = erdbeben.json()["features"][int(beben)]["properties"]["time"]
                   
    mini_dict = {"Ort": place, "Zeit": time, "Stärke": magnitude}
  
    liste.append(mini_dict)

Tabelle = pd.DataFrame(liste)[-10:]

Tabelle.sort_values(by=["Stärke"], ascending = False)

,Ort,Stärke,Zeit
250,Southern East Pacific Rise,4.80,1546302085050
252,"28km ENE of Port-Olry, Vanuatu",4.80,1546301045950
245,"61km SSW of Lithakia, Greece",4.30,1546304596820
249,"69km E of Whittier, Alaska",2.40,1546302737136
246,"82km N of Yakutat, Alaska",1.50,1546303785784
248,"163km E of Chitina, Alaska",1.50,1546302847356
247,"34km NNW of Valdez, Alaska",1.40,1546303645230
253,"35km WSW of Nikiski, Alaska",1.40,1546301040765
251,"23km NNE of Badger, Alaska",1.10,1546301771244
244,"7km WNW of The Geysers, CA",1.06,1546304617740


#### Die überlegene Lösung: Request direkt mit Magnitude ergänzen

In [84]:
erdbeben = requests.get("https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2014-09-24&endtime=2019-09-24&minmagnitude=5.5")
erdbeben = erdbeben.json()
erdbeben.keys()
erdbebenliste = erdbeben['features']

leereliste = []

for erdbeben in erdbebenliste:
    mag = erdbeben['properties']['mag']
    place = erdbeben['properties']['place']
    time = erdbeben['properties']['time']
    
    mini_dict = {"Ort": place, "Zeit": time, "Stärke": mag}
    leereliste.append(mini_dict)
 
Tabelle = pd.DataFrame(leereliste)

Tabelle.sort_values(by=["Stärke"], ascending = False)[:10]

,Ort,Stärke,Zeit
1826,"48km W of Illapel, Chile",8.3,1442444072860
954,"101km SSW of Tres Picos, Mexico",8.2,1504846159180
555,"286km NNE of Ndoi Island, Fiji",8.2,1534637980670
154,"78km SE of Lagunas, Peru",8.0,1558856475073
787,"280km SE of Kodiak, Alaska",7.9,1516699900890
1170,"35km WNW of Panguna, Papua New Guinea",7.9,1485059422960
506,"102km ESE of Suva, Fiji",7.9,1536248958710
1229,"54km E of Taron, Papua New Guinea",7.9,1481971870500
2009,"36km E of Khudi, Nepal",7.8,1429942285950
1551,"27km SSE of Muisne, Ecuador",7.8,1460851116980


5. Wo haben sich diese Erdbeben ereignet?

6. Ziehe auch die Zeit mit heraus?

7. Konvertiere die Unix Zeit zu leserlichen Zeit? (Suche nicht zu lange, schau mal an, wies im Pandas aussieht.)

In [82]:
from datetime import datetime

erdbeben = requests.get("https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2014-01-01&endtime=2019-09-24&minmagnitude=5.5")
erdbeben = erdbeben.json()
erdbeben.keys()
erdbebenliste = erdbeben['features']

leereliste = []

for erdbeben in erdbebenliste:
    mag = erdbeben['properties']['mag']
    place = erdbeben['properties']['place']
    time = str(erdbeben['properties']['time'])[:-3]
    time = datetime.utcfromtimestamp(int(time)).strftime("%Y-%m-%d %H:%M:%S")
   
    mini_dict = {"Ort": place, "Zeit": time, "Stärke": mag}
    leereliste.append(mini_dict)
 
Tabelle = pd.DataFrame(leereliste)

Tabelle = Tabelle.sort_values(by=["Stärke"], ascending = False)[:10]

Tabelle


,Ort,Stärke,Zeit
1826,"48km W of Illapel, Chile",8.3,2015-09-16 22:54:32
2497,"94km NW of Iquique, Chile",8.2,2014-04-01 23:46:47
954,"101km SSW of Tres Picos, Mexico",8.2,2017-09-08 04:49:19
555,"286km NNE of Ndoi Island, Fiji",8.2,2018-08-19 00:19:40
154,"78km SE of Lagunas, Peru",8.0,2019-05-26 07:41:15
1170,"35km WNW of Panguna, Papua New Guinea",7.9,2017-01-22 04:30:22
1229,"54km E of Taron, Papua New Guinea",7.9,2016-12-17 10:51:10
787,"280km SE of Kodiak, Alaska",7.9,2018-01-23 09:31:40
2349,"19km SE of Little Sitkin Island, Alaska",7.9,2014-06-23 20:53:09
506,"102km ESE of Suva, Fiji",7.9,2018-09-06 15:49:18
